# BIZ 프로젝트 : 부실기업 예측

## Step2 : 데이터 전처리

부실기업은 과거 3년간 연속해 이자보상 배율이 1.0미만인 기업을 의미하며  
이를 이용하여 타겟변수인 부실기업여부 변수를 생성

### 필요라이브러리 불러오기

In [51]:
import pandas as pd
import numpy as np

### 데이터 불러오기

In [52]:
RANDOM_STATE = 110

data = pd.read_csv("../../project/data/clean_financial_data.csv", encoding='utf-8-sig', low_memory=False)

In [53]:
data.columns

Index(['업체코드', '종목코드', '종목명', '2020/총자산증가율', '2021/총자산증가율', '2022/총자산증가율',
       '2023/총자산증가율', '2024/총자산증가율', '2020/유형자산증가율', '2021/유형자산증가율',
       ...
       '2020/조세공과(구성비)', '2021/조세공과(구성비)', '2022/조세공과(구성비)', '2023/조세공과(구성비)',
       '2024/조세공과(구성비)', '2020/감가상각비(구성비)', '2021/감가상각비(구성비)',
       '2022/감가상각비(구성비)', '2023/감가상각비(구성비)', '2024/감가상각비(구성비)'],
      dtype='object', length=543)

In [54]:
# '영업이익이자보상비율'이 포함된 열들 찾기
columns_with_ratio = [col for col in data.columns if '영업이익이자보상비율' in col]

# 각 열에 대해 결측치 확인
missing_data = data[columns_with_ratio].isnull().sum()

# 결측치가 있는 열 출력
missing_data[missing_data > 0]

2020/영업이익이자보상비율     8768
2021/영업이익이자보상비율     6378
2022/영업이익이자보상비율     4472
2023/영업이익이자보상비율     4810
2024/영업이익이자보상비율    37943
dtype: int64

- 2024년 데이터는 target변수인 '영업이익이자보상비율'의 대부분이 결측치이므로 제거

In [55]:
# '2024'가 열 이름에 포함된 열들을 제거
data = data.drop(columns=[col for col in data.columns if '2024' in col], errors='ignore')

In [56]:
# '영업이익이자보상비율'이 포함된 열들 찾기
columns_with_ratio = [col for col in data.columns if '영업이익이자보상비율' in col]

# 결측치가 있는 행 제거
data = data.dropna(subset=columns_with_ratio)

# 결과 확인
print(f"남은 데이터 개수: {data.shape[0]} 행")

남은 데이터 개수: 27604 행


In [57]:
# 각 열의 결측값을 중앙값으로 대체
data = data.fillna(data.median(numeric_only=True))

In [58]:
data.dtypes

업체코드                object
종목코드                object
종목명                 object
2020/총자산증가율        float64
2021/총자산증가율        float64
                    ...   
2023/조세공과(구성비)     float64
2020/감가상각비(구성비)    float64
2021/감가상각비(구성비)    float64
2022/감가상각비(구성비)    float64
2023/감가상각비(구성비)    float64
Length: 435, dtype: object

In [59]:
from scipy.stats.mstats import winsorize

# 숫자형 데이터에만 윈저라이징 적용
numeric_columns = data.select_dtypes(include=['number']).columns

for col in numeric_columns:
    # 0.01 이하와 0.99 이상을 해당 값으로 제한
    lower_limit = data[col].quantile(0.01)
    upper_limit = data[col].quantile(0.99)
    data[col] = data[col].clip(lower=lower_limit, upper=upper_limit)


data.head(5)


,업체코드,종목코드,종목명,2020/총자산증가율,2021/총자산증가율,2022/총자산증가율,2023/총자산증가율,2020/유형자산증가율,2021/유형자산증가율,2022/유형자산증가율,...,2022/임차료(구성비),2023/임차료(구성비),2020/조세공과(구성비),2021/조세공과(구성비),2022/조세공과(구성비),2023/조세공과(구성비),2020/감가상각비(구성비),2021/감가상각비(구성비),2022/감가상각비(구성비),2023/감가상각비(구성비)
0,N350605,A000020,동화약품,7.57,2.70,3.27,10.23,19.67,10.41,7.64,...,1.32,1.41,2.14,2.11,2.12,2.13,7.96,7.83,8.03,8.22
1,N320498,A000040,KR모터스,-36.76,1.45,-0.52,-4.89,-0.29,5.78,4.85,...,1.32,1.41,2.14,2.11,2.12,2.13,7.96,7.83,8.03,8.22
2,N320684,A000050,경방,-6.47,0.15,-6.09,-1.40,-36.42,-23.56,665.18,...,1.32,1.41,2.14,2.11,2.12,2.13,7.96,7.83,8.03,8.22
3,N320730,A000070,삼양홀딩스,6.31,22.28,-0.76,17.63,-45.53,943.89,0.68,...,1.32,1.41,2.14,2.11,2.12,2.13,7.96,7.83,8.03,8.22
4,N310581,A000080,하이트진로,-0.99,10.65,-9.43,-0.03,-6.75,-2.60,-1.51,...,1.32,1.41,2.14,2.11,2.12,2.13,7.96,7.83,8.03,8.22


In [60]:
data['2021/영업이익이자보상비율'] < 1

0        False
1         True
2        False
3        False
4        False
         ...  
38634     True
38635    False
38636    False
38637    False
38638    False
Name: 2021/영업이익이자보상비율, Length: 27604, dtype: bool

- 0이면 부실기업이 아닌 기업, 1이면 부실기업을 의미

In [61]:
# 2022/부실기업 변수 생성 및 값 할당
data['2022/부실기업'] = ((data['2020/영업이익이자보상비율'] < 1) & 
                        (data['2021/영업이익이자보상비율'] < 1) & 
                        (data['2022/영업이익이자보상비율'] < 1)).astype(int)

# 2023/부실기업 변수 생성 및 값 할당
data['2023/부실기업'] = ((data['2021/영업이익이자보상비율'] < 1) & 
                        (data['2022/영업이익이자보상비율'] < 1) & 
                        (data['2023/영업이익이자보상비율'] < 1)).astype(int)

# 결과 확인
print("\n2022/부실기업 변수 값:")
print(data['2022/부실기업'].value_counts())

print("\n2023/부실기업 변수 값:")
print(data['2023/부실기업'].value_counts())


2022/부실기업 변수 값:
2022/부실기업
0    22672
1     4932
Name: count, dtype: int64

2023/부실기업 변수 값:
2023/부실기업
0    22473
1     5131
Name: count, dtype: int64


C:\Users\ldb76\AppData\Local\Temp\ipykernel_56672\1223814708.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['2022/부실기업'] = ((data['2020/영업이익이자보상비율'] < 1) &
C:\Users\ldb76\AppData\Local\Temp\ipykernel_56672\1223814708.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['2023/부실기업'] = ((data['2021/영업이익이자보상비율'] < 1) &


In [64]:
data.isnull().sum()

업체코드                   0
종목코드               24402
종목명                    1
2020/총자산증가율            0
2021/총자산증가율            0
                   ...  
2021/감가상각비(구성비)        0
2022/감가상각비(구성비)        0
2023/감가상각비(구성비)        0
2022/부실기업              0
2023/부실기업              0
Length: 437, dtype: int64

In [66]:
# '종목명' 열에서 null 값이 있는 행 제거
data = data[data['종목명'].notnull()]

이때 종목코드 x -> 해당 기업이 상장되지 않은 경우

In [67]:
# '종목코드' 열의 값이 있으면 1, 없으면 0을 부여하는 새로운 열 생성
data['상장여부'] = data['종목코드'].notnull().astype(int)

# '종목코드' 열 제거
data = data.drop(columns=['종목코드'])

C:\Users\ldb76\AppData\Local\Temp\ipykernel_56672\292392295.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['상장여부'] = data['종목코드'].notnull().astype(int)


In [68]:
data.isnull().sum()

업체코드               0
종목명                0
2020/총자산증가율        0
2021/총자산증가율        0
2022/총자산증가율        0
                  ..
2022/감가상각비(구성비)    0
2023/감가상각비(구성비)    0
2022/부실기업          0
2023/부실기업          0
상장여부               0
Length: 437, dtype: int64